In [ ]:
#!/usr/bin/env python
import sys,string,os,math
import time
import shutil
import pyfits
from scipy.interpolate import interp1d
from mirpy import miriad
from matplotlib import pyplot as plt
from matplotlib import rc
from matplotlib import gridspec

import numpy as np

tempoinit=time.time()
print 'LOADED'

#define some fundamental constants

#set fundamental constants
C=2.99792458e5 #km/s
HI=1.420405751e9 #Hz


#define some functions

def SpecExtr(infile,outfile,v_sys,px,py):
 
    hdulist = pyfits.open(infile) #read input
    
    flux=[]
    flux_box_vec=[]
    madfm=[]
    madfm_box=[]

    #read data and header
    scidata = hdulist[0].data
    #reduce dimensions of the input cube
    sci=np.squeeze(scidata)
    prihdr=hdulist[0].header
        
    len_vel=hdulist[0].header['NAXIS3']
    len_pix=hdulist[0].header['NAXIS2']
    #print len_vel
    #x-axis formula
    freq = (np.linspace(1,sci.shape[0],sci.shape[0]) - prihdr['CRPIX3'])* prihdr['CDELT3'] + prihdr['CRVAL3']
    freq = freq/1e3
    
    #v=C*((HI-freq)/freq)

    v=HI/(freq/C+1)
    
    v_wr_sys=(freq-v_sys)
    
    #I determine the pixels where to extract the spectra
    
    #I make the header bi-dimensional so wcs does not get confused
    del prihdr['CTYPE4']
    del prihdr['CDELT4']    
    del prihdr['CRVAL4']
    del prihdr['CRPIX4']
    del prihdr['CTYPE3']
    del prihdr['CDELT3']
    del prihdr['CRVAL3']
    del prihdr['CRPIX3'] 
    
    del prihdr['NAXIS3']
    del prihdr['NAXIS4']
    del prihdr['NAXIS']
    prihdr['NAXIS']=2
    
    #ra=-float(prihdr['CRVAL1'])
    #dec=float(prihdr['CRVAL2'])
    
    #I load the WCS coordinate system:
    #w=wcs.WCS(prihdr)    
    #the third digit gives the origin pixel (either 0 or 1, check with James)
    #px,py=w.wcs_world2pix(ra,dec,0)
    #print px,py
    #extract the spectrum
    
    for i in xrange (0,len_vel):
        pix_x=int(py)
        pix_y=int(px)        
        flux.append(sci[i,pix_x,pix_y])
        #flux_box=np.sum(sci[i,pix_x-3:pix_x+3,pix_y-5:pix_y+5]) # (7X11) box
        #flux_box_vec.append(flux_box)        
        
        #determine the noise as in Whiting 2012 et al. 
        #MADMF: median absolute deviation from the median
        #extract a region were to determine the noise: A BOX 
        #!!!!!!! this box includes also the pixel where we see the line: not so smart !!!!!
        if pix_x != 0 or pix_y !=0:
            rms=np.median(sci[i,pix_x-5:pix_x+5,pix_y-5:pix_y+5])
            med2=abs(sci[i,pix_x,pix_y]-rms)
            madfm.append(np.median(med2)/0.6744888)
        else:
            madfm=np.zeros(sci.shape[0])
        #BOX
        #rms=np.median(sci[i,pix_x-3:pix_x+3,pix_y-5:pix_y+5])
        #med2=np.abs(sci[i,pix_x-3:pix_x+3,pix_y-5:pix_y+5]-rms)
        #madfm_box.append(np.median(med2)/0.6744888)
    
    mean_noise=np.mean(madfm)
    
    #mean_noise_box=np.mean(madfm_box)
    
    
    
    #noise along the spectrum
    noise_lin=(np.std(sci[0:70,pix_x,pix_y])+np.std(sci[151:219,pix_x,pix_y]))/2.
    
    tot=np.column_stack((v,freq,v_wr_sys,flux,madfm))
    #del madfm[:]
    #del med2
    
    #write the spectrum on file
    f=open(outfile,'w')
    f.write('freq[Hz],vel[km/s],vel_rel[km/s],flux[Jy/beam],noise\n')
    np.savetxt(f,tot,delimiter=",",fmt="%s")
    f.close()


    return noise_lin,mean_noise


def plotta(x,y,y2,outfile,flag,flag2,hi_sys,rms_mean):
    

    fig_b = plt.figure(figsize=(12, 8), dpi=100)
    

    
    
    params = {'legend.fontsize': 12,
           'axes.linewidth':2,
            'axes.labelsize':22,
           'lines.linewidth':1,
           'legend.linewidth': 3,
           'xtick.labelsize':22,
           'ytick.labelsize':22,
           'xlabel.fontsize':22,
           'ylabel.fontsize':22,
           'text.usetex': True,
           'text.latex.unicode' : True }
    rc('font',**{'family':'serif','serif':['serif']})
    plt.rcParams.update(params)
    #plt.figtext(0.16,0.8,nome, fontsize=28)
    #plt.figtext(0.6,0.2,r'$v_{sys}$ [MHz]='+str(v),fontsize=14)

    
    
    gs_all_line = gridspec.GridSpec(1, 1)
    
    gs_all_line.update(left=0.1, right=0.9, wspace=0.0,hspace=0.0)

    gs_spec = gridspec.GridSpecFromSubplotSpec(4, 1, subplot_spec=gs_all_line[0], wspace=0.0, hspace=0.0)   
    
    ax_spec = fig_b.add_subplot(gs_spec[0:3, 0])
    ax_noise = fig_b.add_subplot(gs_spec[3, 0]) 
    
    
    ax_spec.plot(x, y, marker='o',color='black',lw=3)
    ax_noise.plot(x, y2, marker=' ',color='black',lw=3)
    
    
 
    xx_spec=[min(x),max(x)]
    yy_spec=[min(y)-max(y)/2.,max(y)+max(y)/2.]

    yy_noise=[min(y2)-max(y2)/4.,max(y2)+max(y2)/4.]

    
    a_spec=[0.0,0.0]
    a_noise=[rms_mean,rms_mean]

    #aa=(HI/vsys-1)*C
    
    
    ax_spec.set_xlim([xx_spec[0],xx_spec[1]])
    ax_noise.set_xlim([xx_spec[0],xx_spec[1]])



    

    
    ax_spec.plot(xx_spec, a_spec,linestyle='--',color='black',lw=3)
    ax_noise.plot(xx_spec, a_noise,linestyle='-.',color='black',lw=3)

    #ax_spec.set_ylabel(r'Flux [mJy]',fontsize=20,labelpad=-4)
    #ax_noise.set_ylabel(r'Noise [mJy]',fontsize=20,labelpad=+3)
    ax_spec.set_xticklabels([ ])

   
    if flag==1:
        ax_spec.set_xlabel(r'Velocity [km s^{-1}]', fontsize=20)
        ax_spec.plot(a_spec,yy_spec,linestyle='--',color='black',lw=3)

    if flag==0:
        b_spec=[hi_sys,hi_sys]

        ax_spec.plot(b_spec,yy_spec,linestyle='--',color='black',lw=3)
        ax_noise.plot(b_spec,yy_noise,linestyle='--',color='black',lw=3)

        ax_spec.set_ylim([-0.018,0.01])
    
        ax_noise.set_ylim([-0.01,0.01])      
        ax_spec.set_xlim([3900,4600])
        ax_noise.set_xlim([3900,4600])
        
        ax_spec.set_ylabel(r'Flux [mJy]',fontsize=20,labelpad=-4)
        ax_noise.set_ylabel(r'Noise [mJy]',fontsize=20,labelpad=+3) 
        ax_noise.set_xlabel(r'Velocity [km s^{-1}]', fontsize=20)
        
        #if flag2==0:
        #    rect1 = patches.Rectangle((xx_spec[0],1.137),xx_spec[1]-xx_spec[0], 0.985, color='grey')
        #    ax_noise.add_patch(rect1)
        #if flag2==1:
        #    rect1 = patches.Rectangle((xx_spec[0],0.882),xx_spec[1]-xx_spec[0], 0.792, color='grey')
        #    ax_noise.add_patch(rect1)
       
    if flag==2:    
        ax_spec.set_ylabel(r'$\tau$', fontsize=20)
        b_spec=[hi_sys,hi_sys]

        ax_spec.plot(b_spec,yy_spec,linestyle='--',color='black',lw=3)
    
        ax_spec.set_xlabel(r'Frequency [$10^{9}$ \,\,{\rm Hz}]', fontsize=20)
    if flag==3:    
        ax_spec.set_ylabel(r'$\tau$', fontsize=20)
        ax_spec.set_xlabel(r'Velocity [km s^{-1}]', fontsize=20)

        ax_spec.plot(a_spec,yy_spec,linestyle='--',color='black',lw=3)
        ax_noise.set_ylabel(r'Noise [$\tau$]',fontsize=20) 

    #plt.show()
    
    fig_b.savefig(outfile,format='pdf',bbox_inches='tight')
    
    return 0



def plotta_over(x1,y1,x2,y2,x3,y3,x_or,y_or,resid_x,resid,outfile,flag,flag2,hi_sys,rms_mean):
    

    fig_b = plt.figure(figsize=(12, 8), dpi=100)
    

    
    
    params = {'legend.fontsize': 12,
           'axes.linewidth':2,
            'axes.labelsize':22,
           'lines.linewidth':1,
           'legend.linewidth': 3,
           'xtick.labelsize':22,
           'ytick.labelsize':22,
           'xlabel.fontsize':22,
           'ylabel.fontsize':22,
           'text.usetex': True,
           'text.latex.unicode' : True }
    rc('font',**{'family':'serif','serif':['serif']})
    plt.rcParams.update(params)
    #plt.figtext(0.16,0.8,nome, fontsize=28)
    #plt.figtext(0.6,0.2,r'$v_{sys}$ [MHz]='+str(v),fontsize=14)

    
    
    gs_all_line = gridspec.GridSpec(1, 1)
    
    gs_all_line.update(left=0.1, right=0.9, wspace=0.0,hspace=0.0)

    gs_spec = gridspec.GridSpecFromSubplotSpec(4, 1, subplot_spec=gs_all_line[0], wspace=0.0, hspace=0.0)   
    
    ax_spec = fig_b.add_subplot(gs_spec[0:3, 0])
    ax_noise = fig_b.add_subplot(gs_spec[3, 0]) 
    
    ax_spec.plot(x_or, y_or, marker='o',color='red',lw=3,label='2013')
 
    ax_spec.plot(x1, y1, marker='o',color='black',lw=3,label='2015 1')
    ax_spec.plot(x2, y2, marker='o',color='blue',lw=3,label='2015 2')
    ax_spec.plot(x3, y3, marker='o',color='green',lw=3,label='2015 3')

    
    ax_spec.legend()
    
    ax_noise.plot(resid_x, resid, marker=' ',color='black',lw=3)
    ax_noise.plot(x2, y2, marker=' ',color='green',lw=3)
    #ax_noise.plot(x3, yn3, marker=' ',color='red',lw=3)
    
    
 
    xx_spec=[min(x1),max(x1)]
    yy_spec=[min(y2)-max(y2)/2.,max(y2)+max(y2)/2.]

    yy_noise=[min(y2)-max(y2)/4.,max(y2)+max(y2)/4.]

    
    a_spec=[0.0,0.0]
    a_noise=[0.0,0.0]

    #aa=(HI/vsys-1)*C
    
    
    ax_spec.set_xlim([xx_spec[0],xx_spec[1]])
    ax_noise.set_xlim([xx_spec[0],xx_spec[1]])



    

    
    ax_spec.plot(xx_spec, a_spec,linestyle='--',color='black',lw=3)
    ax_noise.plot(xx_spec, a_noise,linestyle='-.',color='black',lw=3)

    #ax_spec.set_ylabel(r'Flux [mJy]',fontsize=20,labelpad=-4)
    #ax_noise.set_ylabel(r'Noise [mJy]',fontsize=20,labelpad=+3)
    ax_spec.set_xticklabels([ ])

   
    if flag==1:
        ax_spec.set_xlabel(r'Velocity [km s^{-1}]', fontsize=20)
        ax_spec.plot(a_spec,yy_spec,linestyle='--',color='black',lw=3)

    if flag==0:
        b_spec=[hi_sys,hi_sys]

        #ax_spec.plot(b_spec,yy_spec,linestyle='--',color='black',lw=3)
        #ax_noise.plot(b_spec,yy_noise,linestyle='--',color='black',lw=3)

        ax_spec.set_ylim([-0.02,0.02])
    
        ax_noise.set_ylim([-0.01,0.01])      
        ax_spec.set_xlim([3900,4600])
        ax_noise.set_xlim([3900,4600])
        
        ax_spec.set_ylabel(r'Flux [mJy]',fontsize=20,labelpad=-4)
        ax_noise.set_ylabel(r'Noise [mJy]',fontsize=20,labelpad=+3) 
        ax_noise.set_xlabel(r'Velocity [km s^{-1}]', fontsize=20)
        
        #if flag2==0:
        #    rect1 = patches.Rectangle((xx_spec[0],1.137),xx_spec[1]-xx_spec[0], 0.985, color='grey')
        #    ax_noise.add_patch(rect1)
        #if flag2==1:
        #    rect1 = patches.Rectangle((xx_spec[0],0.882),xx_spec[1]-xx_spec[0], 0.792, color='grey')
        #    ax_noise.add_patch(rect1)
       
    if flag==2:    
        ax_spec.set_ylabel(r'$\tau$', fontsize=20)
        b_spec=[hi_sys,hi_sys]

        ax_spec.plot(b_sepc,yy_spec,linestyle='--',color='black',lw=3)
    
        ax_spec.set_xlabel(r'Frequency [$10^{9}$ \,\,{\rm Hz}]', fontsize=20)
    if flag==3:    
        ax_spec.set_ylabel(r'$\tau$', fontsize=20)
        ax_spec.set_xlabel(r'Velocity [km s^{-1}]', fontsize=20)

        ax_spec.plot(a_spec,yy_spec,linestyle='--',color='black',lw=3)
        ax_noise.set_ylabel(r'Noise [$\tau$]',fontsize=20) 

    #plt.show()
    
    fig_b.savefig(outfile,format='pdf',bbox_inches='tight')
    
    return 0



#EXTRACT the spectrum from the combined dataset

path_or='/Users/maccagni/Sharp_data/PKS/'

os.chdir(path_or)


print '***********First cube*********'



cubefits1='v1/cube_comb.fits'
cube='v1/m_comb'
miriad.fits(IN=cube,out=cubefits1,op='xyout')
outfile='v1/spectrum_comb1.csv'
infile=cubefits1
v_sys=4274
px=512
py=512


print '***********Extract the Spectrum**********'


noise,mean_noise=SpecExtr(infile,outfile,v_sys,px,py)

infile = 'v1/spectrum_comb1.csv'
data=np.genfromtxt(infile,dtype=float, delimiter=',', names=True)
    
#toget the column names
freq1=data['freqHz']
flux1=data['fluxJybeam']
vel1=data['velkms']
vel_rel1=data['vel_relkms']
noise1=data['noise']

mean_noise1=np.mean(noise1)

print '***********Second cube*********'


cubefits2='v2/cube_comb.fits'
cube='v2/m_comb'
miriad.fits(IN=cube,out=cubefits2,op='xyout')
outfile='v2/spectrum_comb2.csv'
infile=cubefits2
v_sys=4274
px=512
py=512


print '***********Extract the Spectrum**********'


noise,mean_noise=SpecExtr(infile,outfile,v_sys,px,py)
infile = 'v2/spectrum_comb2.csv'
data=np.genfromtxt(infile,dtype=float, delimiter=',', names=True)
    
#toget the column names
freq2=data['freqHz']
flux2=data['fluxJybeam']
vel2=data['velkms']
vel_rel2=data['vel_relkms']
noise2=data['noise']

mean_noise2=np.mean(noise1)


print '***********Third cube*********'


cubefits2='v3/cube_comb.fits'
cube='v3/m_comb'
miriad.fits(IN=cube,out=cubefits2,op='xyout')
outfile='v3/spectrum_comb3.csv'
infile=cubefits2
v_sys=4274
px=512
py=512


print '***********Extract the Spectrum**********'


noise,mean_noise=SpecExtr(infile,outfile,v_sys,px,py)
infile = 'v3/spectrum_comb3.csv'
data=np.genfromtxt(infile,dtype=float, delimiter=',', names=True)
    
#toget the column names
freq3=data['freqHz']
flux3=data['fluxJybeam']
vel3=data['velkms']
vel_rel3=data['vel_relkms']
noise3=data['noise']

mean_noise2=np.mean(noise1)




#this loads the cube I reduced in 2013 and extracts the spectrum.
print '***********Original cube**********'


cubefits_or='f.fits'

infile=cubefits_or
outfile='spectrum_or.csv'
noise,mean_noise=SpecExtr(infile,outfile,v_sys,px,py)

infile = 'spectrum_or.csv'
data=np.genfromtxt(infile,dtype=float, delimiter=',', names=True)
    
#toget the column names
freq_or=data['freqHz']
flux_or=data['fluxJybeam']
vel_or=data['velkms']
vel_rel_or=data['vel_relkms']
noise_or=data['noise']

mean_noise_or=np.mean(noise_or)



#outfilefig='spectrum_overlay_han.pdf'

#plotta_over(vel1,flux1,vel2,flux2,vel_or,flux_or,resid_x,resid,outfilefig,int(0),int(0),4274,mean_noise1)


#### PLOTTING and calculating the residuals.

#interpolation to subtract the spectra

path_or='/Users/maccagni/Sharp_data/PKS/'


spec_2 = interp1d(vel3, flux3, kind='cubic',bounds_error=False,fill_value=np.nan)

spec_or = interp1d(vel_or, flux_or, kind='cubic',bounds_error=False,fill_value=np.nan)

resid_x= np.linspace(vel2[0],vel2[-1],1e4,endpoint=True)



resid= spec_or(resid_x) - spec_2(resid_x) 


outfilefig=path_or+'spectrum_overlay_1.pdf'

plotta_over(vel1,flux1,vel2,flux2,vel3,flux3,
            vel_or,flux_or,resid_x,resid,outfilefig,int(0),int(0),4274,0.0)

print 'NORMAL TERMINATION'